Importing Libraries

In [ ]:

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import math
from math import acos, degrees
from scipy import spatial
from scipy.stats import chisquare
from scipy.stats import chi2_contingency
from sklearn.metrics.pairwise import cosine_similarity
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.metrics import plot_confusion_matrix

from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


#Reading in the data as adult dataframe
autism = pd.read_csv("https://raw.githubusercontent.com/rupreti77/upreti_IT533/main/csv_result-Autism-Adult-Data.csv")

#Verifying that we can see the data
autism.head()

,id,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,1,1,1,1,1,0,0,1,1,0,0,26,f,White-European,no,no,United States,no,6,18 and more,Self,NO
1,2,1,1,0,1,0,0,0,1,0,1,24,m,Latino,no,yes,Brazil,no,5,18 and more,Self,NO
2,3,1,1,0,1,1,0,1,1,1,1,27,m,Latino,yes,yes,Spain,no,8,18 and more,Parent,YES
3,4,1,1,0,1,0,0,1,1,0,1,35,f,White-European,no,yes,United States,no,6,18 and more,Self,NO
4,5,1,0,0,0,0,0,0,1,0,0,40,f,?,no,no,Egypt,no,2,18 and more,?,NO


In [ ]:
autism4=autism 
autism4.ethnicity.replace('?','unknown', inplace=True)
autism4.relation.replace('?','unknown', inplace=True)
autism4

,id,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,1,1,1,1,1,0,0,1,1,0,0,26,f,White-European,no,no,United States,no,6,18 and more,Self,NO
1,2,1,1,0,1,0,0,0,1,0,1,24,m,Latino,no,yes,Brazil,no,5,18 and more,Self,NO
2,3,1,1,0,1,1,0,1,1,1,1,27,m,Latino,yes,yes,Spain,no,8,18 and more,Parent,YES
3,4,1,1,0,1,0,0,1,1,0,1,35,f,White-European,no,yes,United States,no,6,18 and more,Self,NO
4,5,1,0,0,0,0,0,0,1,0,0,40,f,unknown,no,no,Egypt,no,2,18 and more,unknown,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,700,0,1,0,1,1,0,1,1,1,1,25,f,White-European,no,no,Russia,no,7,18 and more,Self,YES
700,701,1,0,0,0,0,0,0,1,0,1,34,m,Hispanic,no,no,Mexico,no,3,18 and more,Parent,NO
701,702,1,0,1,1,1,0,1,1,0,1,24,f,unknown,no,no,Russia,no,7,18 and more,unknown,YES
702,703,1,0,0,1,1,0,1,0,1,1,35,m,South Asian,no,no,Pakistan,no,6,18 and more,Self,NO


In [ ]:
autism5=autism4
nan_value = float("NaN")
autism5.replace("?", nan_value, inplace=True)
autism5.dropna(inplace=True)
autism5["age"] = pd.to_numeric(autism5["age"], downcast="float")
autism5.age.describe()
autism5

,id,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,1,1,1,1,1,0,0,1,1,0,0,26.0,f,White-European,no,no,United States,no,6,18 and more,Self,NO
1,2,1,1,0,1,0,0,0,1,0,1,24.0,m,Latino,no,yes,Brazil,no,5,18 and more,Self,NO
2,3,1,1,0,1,1,0,1,1,1,1,27.0,m,Latino,yes,yes,Spain,no,8,18 and more,Parent,YES
3,4,1,1,0,1,0,0,1,1,0,1,35.0,f,White-European,no,yes,United States,no,6,18 and more,Self,NO
4,5,1,0,0,0,0,0,0,1,0,0,40.0,f,unknown,no,no,Egypt,no,2,18 and more,unknown,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,700,0,1,0,1,1,0,1,1,1,1,25.0,f,White-European,no,no,Russia,no,7,18 and more,Self,YES
700,701,1,0,0,0,0,0,0,1,0,1,34.0,m,Hispanic,no,no,Mexico,no,3,18 and more,Parent,NO
701,702,1,0,1,1,1,0,1,1,0,1,24.0,f,unknown,no,no,Russia,no,7,18 and more,unknown,YES
702,703,1,0,0,1,1,0,1,0,1,1,35.0,m,South Asian,no,no,Pakistan,no,6,18 and more,Self,NO


In [ ]:
#Let's drop the value of the age which is equal to 383 at the line 52
autism5.loc[autism5['age']==383]


,id,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
52,53,1,0,0,0,0,0,0,0,0,0,383.0,f,Pasifika,no,no,New Zealand,no,1,18 and more,Self,NO


In [ ]:
autism5=autism5.drop(52)

In [ ]:
autism5.age.describe()

count    701.000000
mean      29.194008
std        9.711530
min       17.000000
25%       21.000000
50%       27.000000
75%       35.000000
max       64.000000
Name: age, dtype: float64

#Selection of the dependent variable in the dataset

In [ ]:
autism5.contry_of_res.replace(['United States','Canada', 'Mexico'],'North_America', inplace=True)
autism5.contry_of_res.replace(['Argentina','Brazil','Bahamas', 'Chile','Costa Rica','Nicaragua','Uruguay','Ecuador','Bolivia','Aruba'],'South_America', inplace=True)
autism5.contry_of_res.replace(['Egypt','Burundi', 'United Kingdom','South Africa','Sierra Leone', 'Ethiopia','Niger','Angola'],'Africa', inplace=True)
autism5.contry_of_res.replace(['Afghanistan','Jordan','Bangladesh','China','India','Lebanon','Philippines','Hong Kong', 'Japan', 'Sri Lanka','Iraq','Viet Nam', 'Malaysia', 'Nepal', 'Indonesia','Turkey','Iran','Saudi Arabia','Azerbaijan','Pakistan','Kazakhstan','Oman','United Arab Emirates'],'Asia', inplace=True)
autism5.contry_of_res.replace([ 'Austria','Spain', 'Ireland', 'United Kingdom','Italy','France','Netherlands', 'Romania', 
                            'Sweden','Germany','Russia','Armenia','Iceland','Ukraine','Serbia', 'Portugal','Belgium',
                            'Finland', 'Czech Republic', 'Cyprus'],'Europe', inplace=True)
autism5.contry_of_res.replace(['New Zealand','Australia', 'Tonga', 'AmericanSamoa',],'Oceania', inplace=True)
autism5.contry_of_res.unique()

array(['North_America', 'South_America', 'Europe', 'Africa', 'Oceania',
       'Asia'], dtype=object)

In [ ]:
autism5.columns = autism5.columns.str.replace('/', '')
autism5
autism5.jundice.replace(('yes', 'no'), (1, 0), inplace=True)
autism5.austim.replace(('yes', 'no'), (1, 0), inplace=True)
autism5.gender.replace (('m', 'f'), (1, 0), inplace=True)
autism5.ClassASD.replace(('YES', 'NO'), (1, 0), inplace=True)
autism6= autism5[['A1_Score','A2_Score','A3_Score','A4_Score','A5_Score','A6_Score','A7_Score','A8_Score','A9_Score','A10_Score','age','jundice','austim','gender' ,'result','contry_of_res']]
autism6

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,jundice,austim,gender,result,contry_of_res
0,1,1,1,1,0,0,1,1,0,0,26.0,0,0,0,6,North_America
1,1,1,0,1,0,0,0,1,0,1,24.0,0,1,1,5,South_America
2,1,1,0,1,1,0,1,1,1,1,27.0,1,1,1,8,Europe
3,1,1,0,1,0,0,1,1,0,1,35.0,0,1,0,6,North_America
4,1,0,0,0,0,0,0,1,0,0,40.0,0,0,0,2,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,0,1,0,1,1,0,1,1,1,1,25.0,0,0,0,7,Europe
700,1,0,0,0,0,0,0,1,0,1,34.0,0,0,1,3,North_America
701,1,0,1,1,1,0,1,1,0,1,24.0,0,0,0,7,Europe
702,1,0,0,1,1,0,1,0,1,1,35.0,0,0,1,6,Asia


In [ ]:
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip # Installing the package first

  Using cached https://github.com/pandas-profiling/pandas-profiling/archive/master.zip


In [ ]:
autism6.describe(include = "all")

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,jundice,austim,gender,result,contry_of_res
count,701.000000,701.000000,701.000000,701.000000,701.000000,701.000000,701.000000,701.000000,701.000000,701.000000,701.000000,701.000000,701.000000,701.000000,701.000000,701
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Asia
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,279
mean,0.723252,0.453638,0.459344,0.497860,0.499287,0.285307,0.417974,0.651926,0.325250,0.574893,29.194008,0.098431,0.129815,0.522111,4.888730,NaN
std,0.447710,0.498201,0.498700,0.500352,0.500357,0.451883,0.493578,0.476700,0.468803,0.494712,9.711530,0.298109,0.336339,0.499868,2.495516,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.000000,0.000000,0.000000,0.000000,3.000000,NaN
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,27.000000,0.000000,0.000000,1.000000,4.000000,NaN
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,35.000000,0.000000,0.000000,1.000000,7.000000,NaN


In [ ]:
autism7.hist()

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f0bc064f8d0>,
      dtype=object)

In [ ]:
autism7 = autism6[['age', 	'jundice' ,	'austim' ,	'gender' ,	'result' ,	'contry_of_res']]
autism7.head()

,age,jundice,austim,gender,result,contry_of_res
0,26.0,0,0,0,6,North_America
1,24.0,0,1,1,5,South_America
2,27.0,1,1,1,8,Europe
3,35.0,0,1,0,6,North_America
4,40.0,0,0,0,2,Africa


In [ ]:
from sklearn.model_selection import train_test_split
x=autism7.iloc[:,:5] 
y=autism7['contry_of_res']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20) 
print("X_train shape: {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))

X_train shape: (560, 5)
X_test shape: (141, 5)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
sev = RandomForestClassifier()
sev.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier()>

In [ ]:
sev.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
y_pred = sev.predict(X_test)
print(y_pred)

['North_America' 'Asia' 'Asia' 'Asia' 'Oceania' 'Asia' 'South_America'
 'North_America' 'North_America' 'South_America' 'Europe' 'North_America'
 'North_America' 'Europe' 'South_America' 'North_America' 'Asia' 'Africa'
 'North_America' 'Asia' 'Asia' 'North_America' 'North_America'
 'North_America' 'Africa' 'Europe' 'Asia' 'Asia' 'North_America' 'Europe'
 'Asia' 'Europe' 'Africa' 'South_America' 'Asia' 'Oceania' 'Africa'
 'Africa' 'Asia' 'Oceania' 'Asia' 'Asia' 'Asia' 'Africa' 'North_America'
 'Asia' 'North_America' 'North_America' 'North_America' 'North_America'
 'Asia' 'Asia' 'Europe' 'Asia' 'Europe' 'North_America' 'Africa' 'Europe'
 'Africa' 'North_America' 'Asia' 'North_America' 'Asia' 'Oceania'
 'Oceania' 'Asia' 'Asia' 'Europe' 'Asia' 'Asia' 'Asia' 'Asia' 'Europe'
 'Oceania' 'North_America' 'Europe' 'Asia' 'Africa' 'Asia' 'North_America'
 'Africa' 'Asia' 'Asia' 'Asia' 'North_America' 'Asia' 'North_America'
 'Asia' 'North_America' 'Asia' 'Asia' 'North_America' 'Oceania' 'Europe'
 '

In [ ]:
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")
np.random.seed(42)
accuracy_score(y_test, y_pred)

0.3475177304964539

In [ ]:

plot_confusion_matrix(sev, X_test, y_test)